## WFCNN train

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader, TensorDataset
import random
import numpy as np
import pandas as pd
import time

from WFCNN import *

global case_study
case_study = 'ITSC'
case_study = 'CWRU'


def train_val_test_split(data, labels, n_samples, split_ratio):
    div1 = round(n_samples*split_ratio)
    div2 = round((n_samples - div1)/2 + div1)
    l = list(range(n_samples))
    if case_study == 'CWRU':
        test_ind_file = 'data_test_ind_CWRU.csv'
    else:
        test_ind_file = 'data_test_ind_ITSC.csv'
    if os.path.isfile(test_ind_file):

      lr = np.loadtxt(test_ind_file, delimiter=",").astype(int)
    else:
      lr = random.sample(l, n_samples)
      np.savetxt(test_ind_file,lr, delimiter=",")

    data_train = np.array([data[idx] for idx in lr[:div1]])
    data_val = np.array([data[idx] for idx in lr[div1:div2]])
    data_test = np.array([data[idx] for idx in lr[div2:]])
    labels_train = np.array([labels[idx] for idx in lr[:div1]])
    labels_val = np.array([labels[idx] for idx in lr[div1:div2]])
    labels_test = np.array([labels[idx] for idx in lr[div2:]])

    return (data_train, data_val, data_test), (labels_train, labels_val, labels_test)
class AugmentedDataset(Dataset):
    def __init__(self, tensors, device):
        self.tensors = tensors
        self.device = device

    def __getitem__(self, index):
        x = torch.from_numpy(np.asarray(self.tensors[0][index][:])).to(self.device)
        y = torch.tensor(
            self.tensors[1][index].astype(np.float32)
        ).to(self.settings[4])
        return x, y

def dataloader_preparation(split_ratio=0.7, batch_size=50):

    global num_classes
    if case_study == 'CWRU':
      datasets = np.loadtxt('Vib_data2.csv',delimiter=",")
      N_s = 2400
      num_classes =12
    else:
      datasets = np.loadtxt('Iq_ITSC_PMSM.csv',delimiter=",")
      N_s = 5000
      num_classes=4
    #N_s=2400

    data = []
    Class_label=np.zeros(len(datasets[:,N_s]))
    for i in range(len(datasets[:,N_s])):#
      x = datasets[i,0:N_s].astype('single')#+np.random.normal(0,.5,N_s).astype('single')
      Class_label[i] = datasets[i,N_s]
    data =  datasets[:,np.newaxis,:N_s]
    labels = Class_label
    # measure the numbers of dataset shape
    n_samples, n_channel, n_length = data.shape
    n_class = len(np.unique(labels))

    # dataloaders
    data_split, labels_split = train_val_test_split(data, labels, n_samples, split_ratio)
    dataloader_train = DataLoader(
        TensorDataset(torch.tensor(data_split[0], dtype=torch.float), torch.tensor(labels_split[0], dtype=torch.long)),
        batch_size=batch_size,
        shuffle=True,
    )

    dataloader_val = DataLoader(
        TensorDataset(torch.tensor(data_split[1], dtype=torch.float), torch.tensor(labels_split[1], dtype=torch.long)),
        batch_size=batch_size,
    )

    dataloader_test = DataLoader(
        TensorDataset(torch.tensor(data_split[2], dtype=torch.float), torch.tensor(labels_split[2], dtype=torch.long)),
        batch_size=batch_size,
    )

    return dataloader_train, dataloader_val, dataloader_test

def log_results(log_name, results):
    df = pd.DataFrame(results, columns=['loss_train', 'acc_train', 'loss_val', 'acc_val','train_time', 'test_time'])
    df.to_csv(log_name)

def save_model(best_acc, epoch, model):
    print('--------> The best model has been replaced.')
    print('epoch: '+str(epoch)+' | best_acc: '+str(best_acc))
    model_path = './wdcnn2d_epoch_'+case_study+str(epoch)+'pytorch.pt'
    torch.save(model.state_dict(), model_path)
    print('The best model has been saved in '+model_path)

def train(dataloader, device, model, criterion, optimizer):
    total_loss = 0.0
    total_corrects = 0.0
    n_train = 0
    model.train()
    model.zero_grad()

    for batch_idx, batch in enumerate(dataloader):
        input, label = tuple(t.to(device) for t in batch)
        label = label.long()
        output = model(input)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        total_corrects += pred.eq(label.data.view_as(pred)).cpu().sum()
        total_loss += loss.item()
        n_train += label.size()[0]

    epoch_loss = total_loss / n_train
    epoch_acc = 100. * total_corrects.numpy() / n_train
    print('Train loss: '+str(epoch_loss)+' || Train accuracy: '+str(epoch_acc))
    return epoch_loss, epoch_acc

def test(dataloader, device, model, criterion):
    total_loss = 0.0
    total_corrects = 0.0
    n_test = 0
    model.eval()
    model.zero_grad()

    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            input, label = tuple(t.to(device) for t in batch)
            label = label.long()
            model=model.to(device)
            output = model(input.double())
            loss = criterion(output, label)
            pred = output.data.max(1, keepdim=True)[1]
            total_corrects += pred.eq(label.data.view_as(pred)).cpu().sum()
            total_loss += loss.item()
            n_test += label.size()[0]
    epoch_loss = total_loss / n_test
    epoch_acc = 100.0 * total_corrects.numpy() / n_test
    print('Test loss: '+str(epoch_loss)+' || Test accuracy: '+str(epoch_acc))
    return epoch_loss, epoch_acc

if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #torch.cuda.set_per_process_memory_fraction(1.0, device)

    log_file_name = 'results_logged_'+case_study+'_WFCNN.csv'
    n_epoch = 500
    batch = 14

    dataloader_train, dataloader_val, dataloader_test = dataloader_preparation(batch_size=batch)
    model = Net(n_class=num_classes,
                   case_study = case_study,
                  )


    model.to(device)
    model=model.double()
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    best_acc = 0.0
    results = np.zeros([n_epoch,6])
    for epoch in range(n_epoch):
        print('------------------ epoch '+str(epoch)+' ------------------')
        model=model.double()
        model.eval()
        t0 = time.time()
        loss_train, acc_train = train(dataloader_train, device, model, criterion, optimizer)
        t1 = time.time()
        train_time= t1-t0
        print('Training time is:'+str(train_time))
        loss_val, acc_val = test(dataloader_val, device, model, criterion)
        test_time = time.time()-t1
        print('Test time is:'+str(test_time))
        results[epoch,:] = np.array([loss_train, acc_train, loss_val, acc_val, train_time, test_time])
        log_results(log_file_name,results)

        # save better model
        if best_acc <= acc_val:
            best_acc = acc_val
            loss_test, acc_test = test(dataloader_test, device, model, criterion)

            if acc_train==100 and acc_val==100:
              model_path = './WFCNN_epoch_'+case_study+str(epoch)+'.pt'
              torch.save(model.state_dict(), model_path)
              print('The model saved: '+model_path)


C:\Users\artaa\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


------------------ epoch 0 ------------------
Train loss: 0.17642243453818673 || Train accuracy: 8.957528957528957
Training time is:0.8056843280792236
Test loss: 0.17364898156511388 || Test accuracy: 14.414414414414415
Test time is:0.04993128776550293
Test loss: 0.17232857397480514 || Test accuracy: 15.495495495495495
------------------ epoch 1 ------------------
Train loss: 0.16891559654258786 || Train accuracy: 24.44015444015444
Training time is:0.6167964935302734
Test loss: 0.16364901052476258 || Test accuracy: 31.17117117117117
Test time is:0.07854557037353516
Test loss: 0.1612131593653615 || Test accuracy: 34.234234234234236
------------------ epoch 2 ------------------
Train loss: 0.1566729520115162 || Train accuracy: 32.35521235521235
Training time is:0.6269869804382324
Test loss: 0.15081471273329736 || Test accuracy: 36.75675675675676
Test time is:0.04999995231628418
Test loss: 0.14724309162637403 || Test accuracy: 37.65765765765766
------------------ epoch 3 ------------------

#model summary

In [5]:
from torchsummary import summary
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
summary(model.double(),(1,2400))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1              [-1, 16, 297]              32
            Conv1d-2              [-1, 16, 149]           1,280
       BatchNorm1d-3              [-1, 16, 149]              32
       ConvBlock1d-4              [-1, 16, 149]               0
         MaxPool1d-5               [-1, 16, 74]               0
            Conv1d-6               [-1, 16, 74]           1,280
       BatchNorm1d-7               [-1, 16, 74]              32
       ConvBlock1d-8               [-1, 16, 74]               0
         MaxPool1d-9               [-1, 16, 19]               0
           Conv2d-10             [-1, 1, 14, 9]              10
          Flatten-11                  [-1, 126]               0
           Linear-12                   [-1, 12]           1,524
Total params: 4,190
Trainable params: 4,190
Non-trainable params: 0
-----------------------------------

In [ ]:
from torchsummary import summary
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
summary(model.double(),(1,5000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1              [-1, 16, 619]              32
            Conv1d-2              [-1, 16, 310]           1,280
       BatchNorm1d-3              [-1, 16, 310]              32
        BasicBlock-4              [-1, 16, 310]               0
         MaxPool1d-5              [-1, 16, 154]               0
            Conv1d-6              [-1, 16, 154]           1,280
       BatchNorm1d-7              [-1, 16, 154]              32
        BasicBlock-8              [-1, 16, 154]               0
         MaxPool1d-9               [-1, 16, 39]               0
           Conv2d-10            [-1, 1, 14, 37]              10
          Flatten-11                  [-1, 518]               0
           Linear-12                    [-1, 4]           2,076
Total params: 4,742
Trainable params: 4,742
Non-trainable params: 0
-----------------------------------